In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:


img_size = 224
batch_size = 128

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                rotation_range=20,
                                                                width_shift_range=0.2,
                                                                height_shift_range=0.2,
                                                                shear_range=0.2,
                                                                zoom_range=0.2,
                                                                horizontal_flip=True)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_ds = train_datagen.flow_from_directory(
    "train",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = True
)
val_ds = validation_datagen.flow_from_directory(
    "valid",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False
)


Found 13492 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


In [6]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Rescaling, Flatten
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

model = keras.Sequential()

model.add(Conv2D(64, (3,3), 1, activation='relu', input_shape=(224, 224, 3)))
model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3,3), 1, activation='relu'))
model.add(Conv2D(256, (3,3), 1, activation='relu'))
model.add(Conv2D(256, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3,3), 1, activation='relu'))
model.add(Conv2D(512, (3,3), 1, activation='relu'))
model.add(Conv2D(512, (3,3), 1, activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(0.0001),
              metrics=['accuracy'])

In [5]:
epochs = 100

es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20, restore_best_weights=True)

h = model.fit(
        train_ds,
        epochs=epochs, 
        validation_data=val_ds,
        callbacks = [es]
)

Epoch 1/100
106/106 [==============================] - 93s 868ms/step - loss: 2.2371 - accuracy: 0.5437 - val_loss: 2.0896 - val_accuracy: 0.5780
Epoch 2/100
106/106 [==============================] - 94s 883ms/step - loss: 2.2071 - accuracy: 0.5517 - val_loss: 2.2207 - val_accuracy: 0.5540
Epoch 3/100
106/106 [==============================] - 94s 884ms/step - loss: 2.1860 - accuracy: 0.5566 - val_loss: 2.1101 - val_accuracy: 0.5880
Epoch 4/100
106/106 [==============================] - 94s 880ms/step - loss: 2.2000 - accuracy: 0.5449 - val_loss: 2.2556 - val_accuracy: 0.5680
Epoch 5/100
106/106 [==============================] - 94s 882ms/step - loss: 2.1953 - accuracy: 0.5528 - val_loss: 2.2099 - val_accuracy: 0.5900
Epoch 6/100
106/106 [==============================] - 94s 884ms/step - loss: 2.2013 - accuracy: 0.5560 - val_loss: 2.1194 - val_accuracy: 0.6060
Epoch 7/100
106/106 [==============================] - 94s 882ms/step - loss: 2.1803 - accuracy: 0.5548 - val_loss: 2.1323 -

KeyboardInterrupt: 

In [10]:
model.save('./saved_model/my_model_73_best')

INFO:tensorflow:Assets written to: ./saved_model/my_model_73_best\assets


INFO:tensorflow:Assets written to: ./saved_model/my_model_73_best\assets


In [1]:
import tensorflow as tf
from tensorflow import keras
import os
from PIL import Image
import numpy as np
from skimage import transform

lista_deportes = os.listdir("./test")


def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (224, 224, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

modelo = keras.models.load_model("./saved_model/my_model_73_best")

bmx1 = load("./pruebas/7.jpg")

y_proba = modelo.predict(bmx1)

print(lista_deportes[np.argmax(y_proba)])

1/1 [==============================] - 5s 5s/step
bull riding


In [2]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory("test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle = False)

modelo.evaluate(test_generator)

Found 500 images belonging to 100 classes.
16/16 [==============================] - 5s 181ms/step - loss: 1.1740 - accuracy: 0.7540


[1.1739875078201294, 0.7540000081062317]